We start by adding the necessary folders to the current working path.

In [1]:
# import sys/os
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-4])

# add utils folder to current working path
sys.path.append(parent_directory+"/subfunctions/utils")

# add integration folder to current working path
sys.path.append(parent_directory+"/subfunctions/integration")

# add FTLE folder to current working path
sys.path.append(parent_directory+"/demos/AdvectiveBarriers/FTLE3D")

# add ActiveBarriers folder to current working path
sys.path.append(parent_directory+"/demos/ActiveBarriers")

# Overview

We illustrate the numerical implementation and the use of the active Finite Time Lyapunov Exponent ($ \mathrm{aFTLE} $) on a direct numerical simulation of a turbulent channel flow from the publicly available [Johns Hopkins University Turbulence
Database (JHTDB)](http://turbulence.pha.jhu.edu/). The active LCS diagnostic is applied to the three-dimensional and steady instantaneous linear momentum barrier equation. The notebook is structured as follows:

1. Import Laplacian of velocity-field data stored in the folder 'data/Turbulence'. The Laplacian of the velocity field from the publicly available John Hopkins University Turbulence Database has been [precomputed](../../../../data/Turbulence/Channel.ipynb).
<br />
2. Define computational parameters (such as the number of cores) and data.
 <br />
3. Define spatio-temporal domain.
 <br />
4. Interpolate Laplacian of velocity from (discrete) gridded data.
 <br />
5. Instantaneous linear momentum barrier equation active Finite Time Lyapunov Exponent ($ \mathrm{aFTLE} $):

   <br />
   * For 3D incompressible, uniform-density Navier–Stokes flows, the Eulerian barrier equation for linear momentum is:
   <br />
   \begin{equation}
   \mathbf{x'}(s) = \nu \rho \Delta \mathbf{v}(\mathbf{x}, t)
   \label{eq: linearmoment1}
   \end{equation}
   
   <br />
   
   By appropriate rescaling with respect to the dummy time, equation \ref{eq: linearmoment1} simplifies to: 
   <br />
   \begin{equation}
   \mathbf{x'}(s) = \Delta \mathbf{v}(\mathbf{x}, t)
   \label{eq: linearmoment2}
   \end{equation}
   
    Time 't' is just a fixed parameter and hence the system is autonomous and the integration is done with respect to a dummy time variable 's'. 's' can be arbitrarily large and the $ \mathrm{aFTLE} $ is only limited by the spatial resolution and domain. It can thus potentially reveal finer and more resolved structures compared to the classic version of the $ \mathrm{FTLE} $, which is fundamentally limited by the temporal domain of the available data.
    
    <br />
    
   * We start by evaluating the right-hand-side of equation \ref{eq: linearmoment2} by first interpolating $ \Delta \mathbf{v}(\mathbf{x}, t) $ over the spatial meshgrid.
   
   <br />
   
   * We then apply the $ \mathrm{aFTLE} $ to the autonomous ODE from equation \eqref{eq: linearmoment2} over different dummy time-intervals.
 <br />
6. References

# Import data

In [ ]:
# import scipy
import scipy.io as sio

# import numpy
import numpy as np

f = sio.loadmat('../../../../data/Turbulence/Channel.mat')

lap_U = f['Lap_u'] # array (NY, NX, NZ)
lap_V = f['Lap_v'] # array (NY, NX, NZ)
lap_W = f['Lap_w'] # array (NY, NX, NZ)

x = f['xspan'] # array (1, NX)
y = f['yspan'] # array (1, NY)
z = f['zspan'] # array (1, NZ)

# Computational parameters and data

Here we define the computational parameters and the data.

In [ ]:
# import numpy
import numpy as np

# Number of cores for parallel computing
Ncores = 7 # int

# Periodic boundary conditions
periodic_x = False # bool
periodic_y = False # bool
periodic_z = False # bool
periodic = [periodic_x, periodic_y, periodic_z]

# Unsteady velocity field
bool_unsteady = False # bool

# Defined domain
defined_domain = np.isfinite(lap_U).astype(int) # array (NY, NX, NZ)

## Compute meshgrid of dataset
X, Y, Z = np.meshgrid(x, y, z) # array (NY, NX), array (NY, NX, NZ)

## Resolution of meshgrid
dx_data = X[0,1,0]-X[0,0,0] # float
dy_data = Y[1,0,0]-Y[0,0,0] # float
dz_data = Y[0,0,1]-Z[0,0,0] # float

delta = [dx_data, dy_data, dx_data] # list (3, )

# Spatio-temporal domain

Here we define the spatio-temporal domain over which to consider the dynamical system.

In [ ]:
# boundaries
xmin = 3.6 # float
xmax = 4 # float
ymin = 0.2 # float
ymax = 0.6 # float
z_section = 2.55 # float

# Resolution of meshgrid
Ny = 400 # int
Nx = 400 # int

x_domain = np.linspace(xmin, xmax, Nx, endpoint = True) # array (Nx, )
y_domain = np.linspace(ymin, ymax, Ny, endpoint = True) # array (Ny, )

dx = x_domain[1]-x_domain[0] # float
dy = y_domain[1]-y_domain[0] # float

X_domain, Y_domain = np.meshgrid(x_domain, y_domain) # array (Ny, Nx)

# auxiliary grid spacing
aux_grid = [0.1*dx, 0.1*dy, 0.1*dy] # list (3,)

# Define dummy time-intervals for computation of aFTLE in ascending order.
s1 = 10**(-4) # float s1 > 0
s2 = 0.5*10**(-3) # float s2 > s1
s3 = 10**(-3) # float s3 > s2

s = [s1, s2, s3] # list (3,)

# define dummy time interval
ds = .5*10**(-5)
sN = s[-1] # Feel free to change this parameter as needed
s_dummy = np.arange(0, sN+ds, ds)

# Interpolate Laplacian of velocity

In order to evaluate the laplacian of the velocity field at arbitrary locations, we interpolate the gridded laplacian of velocity data. The interpolation with respect to time and space is "linear".

In [ ]:
# Import interpolation function for steady flow field
from ipynb.fs.defs.Interpolant import interpolant_steady

# Set nan values to zero (in case there are any) so that we can apply interpolant. 
# Interpolant does not work if the array contains nan values. 
lap_U[np.isnan(lap_U)] = 0
lap_V[np.isnan(lap_V)] = 0
lap_W[np.isnan(lap_W)] = 0

# Interpolate velocity data using cubic spatial interpolation
Interpolant = interpolant_steady(X, Y, Z, lap_U, lap_V, lap_W)

Interpolant_lap_u = Interpolant[0] # RectangularBivariateSpline-object
Interpolant_lap_v = Interpolant[1] # RectangularBivariateSpline-object
Interpolant_lap_w = Interpolant[2] # RectangularBivariateSpline-object

# Instantaneous linear momentum barriers

We visualize linear momentum/vorticity barriers by applying the $ \mathrm{FTLE} $ to trajectories of instantaneous linear momentum/vorticity barrier equation \ref{eq: linearmoment2}. As soon as the $ \mathrm{FTLE} $-field is applied to an active vector field, such as linear momentum, we refer to it as $ \mathrm{aFTLE} $ (active $ \mathrm{FTLE} $).

## active FTLE

We now compute the active $ \mathrm{FTLE} $ for the instantaneous linear momentum/vorticity barrier equations. First, we evaluate the gradient of the active flow map over the meshgrid. The active flow map is given by the solution of the autonomous ODE from equation \ref{eq: linearmoment2}. Finally, we apply the classic $ \mathrm{FTLE} $ to the gradient of the active flow map. The computation of the active $ \mathrm{FTLE} $ is not done over the 3D meshgrid but only over the 2D cross-section.

In [ ]:
# Import function to compute gradient of flow map
from ipynb.fs.defs.gradient_flowmap import gradient_flowmap

# Import function to compute finite time Lyapunov exponent (FTLE)
from ipynb.fs.defs.FTLE import _FTLE

# Import package for parallel computing
from joblib import Parallel, delayed

def compute_aFTLE(x0, y0, z0, s_dummy, s):
    
    X0 = np.array([x0, y0, z0]) # array (3, Nx*Ny*Nz)
    
    activeDF = gradient_flowmap(s_dummy, X0, X, Y, Z, Interpolant_lap_u, Interpolant_lap_v, Interpolant_lap_w, periodic, bool_unsteady, aux_grid) # array (Nt, 3, 3, Nx*Ny*Nz)
    
    aFTLE = np.zeros((len(s), activeDF.shape[-1]))
    
    counter = 0
    for idx in np.searchsorted(s_dummy, s):
        
        for i in range(activeDF.shape[3]):
            
            aFTLE[counter, i] = _FTLE(activeDF[idx,:,:,i], s_dummy[idx])
            
        counter += 1
    
    return aFTLE

# Split x0,y0,z0 into 'Ncores' equal batches for parallel computing
def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))

In [ ]:
# Vectorize initial conditions by arranging them to a vector of size (Nx*Ny*Nz, )
x0 = X_domain.ravel() # array (Nx*Ny, )
y0 = Y_domain.ravel() # array (Nx*Ny, )
z0 = z_section*np.ones(x0.shape[0],) # array (Nx*Ny, )

x0_batch = list(split(x0, Ncores)) # list (Nx*Ny)
y0_batch = list(split(y0, Ncores)) # list (Nx*Ny)
z0_batch = list(split(z0, Ncores)) # list (Nx*Ny)
results = Parallel(n_jobs=Ncores, verbose = 2)(delayed(compute_aFTLE)(x0_batch[i], y0_batch[i], z0_batch[i], s_dummy, s) for i in range(len(x0_batch)))

# Extract results of aFTLE computation on xy-plane
aFTLE = results[0]
for res in results[1:]:
    aFTLE = np.append(aFTLE, res, axis = -1)
    
X0 = np.array(x0).reshape(Ny,Nx) # array (Ny, Nx)
Y0 = np.array(y0).reshape(Ny,Nx) # array (Ny, Nx)
Z0 = np.array(z0).reshape(Ny,Nx) # array (Ny, Nx)
aFTLE = np.array(aFTLE).reshape(len(s),Ny,Nx) # array (3, Ny, Nx)

##### aFTLE over different dummy time-intervals

In the following we visualize the instantaneous linear momentum/vorticity barriers at time $ t_0 = 0 $ through the $ \mathrm{aFTLE} $ over different dummy time-intervals. Ridges of $ \mathrm{aFTLE}_{0,0}^{s_i} \ (i \in \lbrace 1,2,3 \rbrace) $ indicate transport barriers which block the transport of linear momentum. Note that contrary to the classic $ \mathrm{FTLE} $, the $ \mathrm{aFTLE} $ diagnostic is not limited by the temporal domain as the dummy time-interval can be chosen to be arbitrarily large. We explore the structures of the $ \mathrm{aFTLE}_{0,t_0}^{s_i} $ over different dummy time-intervals $ s_i \in \lbrace 1,2,3 \rbrace $ at time $ t_0 $.

In [ ]:
# Import matplotlib
import matplotlib.pyplot as plt

# plot iterate over different dummy time-intervals and display aFTLE field calculated over each time-interval
def plot_aFTLE(i):
    
    # create figure
    fig = plt.figure(dpi = 350)
    ax = plt.axes()
    
    # Fontsize
    font_size = 7
    
    # Title
    ax.set_title(r'$ \mathrm{aFTLE}$'+f'$_{{{0}}}^{{{s[i]}}}$', fontsize = font_size)
    
    # Contourplot
    cax = ax.contourf(X_domain, Y_domain, aFTLE[i,:,:], levels = 400, cmap = "gist_rainbow_r")
    
    # Set labels
    ax.set_xlabel("x", fontsize = font_size)
    ax.set_ylabel("y", fontsize = font_size)
    
    # Set ticks
    ax.set_xticks(np.arange(xmin, xmax+0.01, 0.1))
    ax.set_yticks(np.arange(ymin, ymax+0.01, 0.1))
    
    # Set fontsize
    ax.xaxis.set_tick_params(labelsize=font_size)
    ax.yaxis.set_tick_params(labelsize=font_size)
    
    # Colorbar
    cbar = plt.colorbar(cax, format='%.0e')
    cbar.ax.tick_params(labelsize=font_size)
    ax.set_aspect('equal')
    plt.show()

##### $ \mathrm{aFTLE}_{0,0}^{s_1} $

Instantaneous linear momentum barriers at time $ t_0 = 0 $ from $ \mathrm{aFTLE}_{0,0}^{s_1} $.

In [ ]:
################################################ PLOT RESULTS ################################################
plot_aFTLE(0)

##### $ \mathrm{aFTLE}_{0,0}^{s_2} $

Instantaneous linear momentum barriers at time $ t_0 = 0 $ from $ \mathrm{aFTLE}_{0,0}^{s_2} $.

In [ ]:
################################################ PLOT RESULTS ################################################
plot_aFTLE(1)

##### $ \mathrm{aFTLE}_{0,0}^{s_3} $

Instantaneous linear momentum barriers at time $ t_0 = 0 $ from $ \mathrm{aFTLE}_{0,0}^{s_3} $.

In [ ]:
################################################ PLOT RESULTS ################################################
plot_aFTLE(2)

The figures focus on momentum-based active hyperbolic barriers in a section of a 3D turbulent channel flow. In 2D cross-sections of the flow, hyperbolic regions which inhibit the transport of linear momentum can be located as ridges of the $ \mathrm{aFTLE} $-field. The $ \mathrm{aFTLE} $ provides a clear picture of active hyperbolic barriers. The features become even more pronounced for longer s-times, revealing several small scale hyperbolic structures. As the turbulent channel flow is spatially limited, one must be aware that by increasing the dummy time interval, trajectories might leave the domain where the velocity field (and hence also the laplacian of the velocity field) is well defined. Furthermore, it must be taken into account that increasing the dummy time horizon, also emphasizes numerical errors and hence the $ \mathrm{aFTLE} $ become more sensitive to noise.

# References

[1] Haller, G., Katsanoulis, S., Holzner, M., Frohnapfel, B., & Gatti, D. (2020). Objective barriers to the transport of dynamically active vector fields. Journal of Fluid Mechanics, 905.

[2] Notebook 9.7.2. in "Transport Barriers and Coherent Structures in Flow Data" by Prof. George Haller.